<a href="https://colab.research.google.com/github/nyoons/ESAA/blob/main/Ch07_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Ch 07. 앙상블 학습과 랜덤 포레스트**
앙상블 학습, 앙상블 방법 = 대중의 지혜

ex. 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킨다. 예측을 하려면 모든 개별 트리의 예측을 구하면 된다. 가장 많은 선택을 받은 클래스를 예측으로 삼는다. 결정 트리 앙상블 = 랜덤 포레스트.

배깅, 부스팅, 스태킹 등을 알아보자.

##7.1 투표 기반 분류기
정확도가 80%인 분류기 여러 개를 훈련시켰다고 하자. 더 좋은 분류기를 만드는 간단한 방법은 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것. 이런 다수결을 **직접 투표** 분류기라고 한다. 이 다수결 분류기가 개별 분류기 중 가장 뛰어난 것보다 정확도가 높은 경우가 많다.

하지만 모든 분류기가 완벽히 독립적이로 오차에 상관관계가 없어야 가능하다. 같은 데이터로 훈련시키는 경우 이런 가정이 맞지 않다.

여러 분류기를 조합해 사이킷런의 투표 기반 분류기를 만들고 훈련시키자.

In [4]:
#데이터 먼저 불러오자
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y=make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

voting_clf=VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [6]:
#각 분류기의 테스트셋 정확도를 확인하자.
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.9
SVC 0.9
VotingClassifier 0.9


예상대로 투표 기반 분류기가 다른 개별 분류기보다 성능이 조금 더 높다.

모든 분류기가 클래스의 확률을 예측할 수 있으면 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다. **간접 투표**. 확률이 높은 투표에 비중을 두기 때문에 직접 투표 방식보다 성능이 높다. voting='soft'로 바꾸어 사용한다. SVC는 기본값에서는 클래스 확률을 제공하지 않으므로 probability=True 지정해야한다.

##7.2 배깅과 페이스팅
다양한 분류기를 만드는 다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것이다. 훈련 세트에서 중복을 허용하여 샘플링하면 **배깅**, 중복 허용 안하면 **페이스팅**이다.

모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만든다. 분류일 때는 최빈값, 회귀에 대해서는 평균. 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하나 분산은 줄어든다.

###7.2.1 사이킷런의 배깅과 페이스팅
결정 트리 분류기 500개의 앙상블을 훈련시켜보자.

In [9]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred=bag_clf.predict(X_test)

여기서 페이스팅을 사용하고 싶다면 bootstrap=False 로 설정하면 된다.

단일 결정 트리와 500개 트리로 만든 배깅 앙상블을 비교한 그림을 보자. 앙상블의 예측이 일반화가 훨씬 잘된 것 같다. 비슷한 편향에서 더 작은 분산을 만든다.

부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높다. 하지만 다양성을 추가한다는 것은 앙상블의 분산을 감소시킬 수 있다!

###7.2.2 oob 평가
배깅을 사용하면 어떤 샘플은 여러번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있다. 선택되지 않은 훈련 샘플의 나머지(37% 정도)를 oob 샘플이라고 부른다. 예측기마다 이 남겨진 샘플은 모두 다르다.

예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob 샘플을 사용해 평가할 수 있다. 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻는다.

oob_score=True로 설정해 자동으로 oob 평가를 수행해보자.

In [10]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9375

oob 평가로는 93.75%의 정확도를 얻을 것으로 보인다.

In [11]:
from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9

테스트 세트에서 90% 정확도를 얻었다. 매우 비슷.

oob 샘플에 대한 결정 함수의 값도 확인할 수 있다. 이 경우 결정 함수는 각 훈련 샘플의 클래스 확률을 반환한다.

In [12]:
bag_clf.oob_decision_function_

array([[0.        , 1.        ],
       [0.825     , 0.175     ],
       [0.16666667, 0.83333333],
       [0.52      , 0.48      ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.02247191, 0.97752809],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.05172414, 0.94827586],
       [0.9895288 , 0.0104712 ],
       [0.        , 1.        ],
       [0.21764706, 0.78235294],
       [0.        , 1.        ],
       [0.01183432, 0.98816568],
       [0.81920904, 0.18079096],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.87777778, 0.12222222],
       [1.        , 0.        ],
       [0.00534759, 0.99465241],
       [0.27567568, 0.72432432],
       [0.30726257, 0.69273743],
       [0.99492386, 0.00507614],
       [1.        , 0.        ],
       [0.59278351, 0.40721649],
       [0.

##7.3 랜덤 패치와 랜덤 서브스페이스
BaggingClassifier는 특성 샘플링도 지원한다. 샘플이 아니고 특성에 대한 샘플링이라 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련된다.

매우 고차원의 데이터셋을 다룰 때 유용하다. 훈련 특성과 샘플을 모두 샘플링하는 것을 **랜덤 패치 방식**이라고 한다. 훈련 샘플을 모두 사용하고, 특성은 샘플링하는 것을 **랜덤 서브스페이스 방식**이라고 한다.

특성 샘플링은 편향은 늘리지만 분산은 낮춘다.

##7.4 랜덤 포레스트
랜덤 포레스트는 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블.

최대 16개 리프 노드를 갖는 500개 트리로 이뤄진 랜덤 포레스트 분류기를 훈련시켜보자.

In [13]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf=rnd_clf.predict(X_test)

랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위성을 더 주입한다. 트리를 다양하게 만들고 편향을 손해보는 대신 분산을 낮춰 전체적으로는 더 훌륭한 모델이 만들어진다.

In [14]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16), n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

###7.4.1 엑스트라 트리
랜덤 포레스트에서 트리를 만들 때 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용한다. 극단적으로 무작위한 트리의 랜덤 포레스트를 **익스트림 랜덤 트리=엑스트라 트리**라고 한다. 역시 편향 늘지만 분산 낮춘다. 일반적인 랜덤 포레스트보다 훨씬 빠르기도 하다. ExtraTreesClassifier를 사용하며 사용법은 랜던 포레스트와 같다.

###7.4.2 특성 중요도
랜덤 포레스트의 다른 장점은 특성의 상대적 중요도를 측정하기 쉽다는 것. 사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정한다. 훈련이 끝나면 특성마다 자동으로 이 점수를 계산해 중요도의 합이 1이 되도록 결과값을 정규화한다. -> feature_importance_ 변수에 저장됨.

iris data에 랜덤 포레스트 훈련시키고 특성의 중요도 출력해보자.

In [15]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09217893797618451
sepal width (cm) 0.022176184262616336
petal length (cm) 0.4505526768589856
petal width (cm) 0.4350922009022135


랜덤 포레스트를 이처럼 특성을 선택해야 할 때 어떤 특성이 중요한지 빠르게 확인할 수 있다.

##7.5 부스팅
약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블. 앞의 모델을 보완해나가며 일련의 예측기를 학습한다. 에이다부스트와 그레디언트 부스트가 인기.

###7.5.1 에이다부스트
이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 높이는 것 -> 에이다부스트.

첫 번째 분류기를 훈련 세트에서 훈련시키고 예측을 만든다. 이 알고리즘이 잘못 분류한 훈련 샘플의 가중치를 상대적으로 높인다. 두번째 분류기는 이 가중치를 사용해 훈련세트에서 훈련하고 다시 예측을 만든다. 그 다음 다시 가중치를 업데이트하는 식으로 계속된다. 이런 연속된 기법은 경사 하강법(계속 한 예측기의 모델 파라미터를 조정해감)과 비슷한 면이 있다.

모든 예측기가 훈련을 마치면 이 앙상블은 배깅이나 페이스팅과 비슷한 방식으로 예측을 만든다. 하지만 예측기마다 다른 가중치가 적용된다.

예측기가 정확할수록 가중치(aj)가 더 높아지게 될 것. 가중치를 업데이트하면 모든 샘플의 가중치를 정규화한다. 과정을 반복하다가 지정된 예측기 수에 도달하거나 완벽한 예측기가 만들어지면 중단된다.

200개의 아주 얕은 결정 트리를 기반으로 하는 에이다부스트를 훈련시켜보자.

In [16]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

###7.5.2 그레디언트 부스팅
에이다부스트처럼 앙상블에 이전 오차를 보정하도록 예측기를 순차적으로 추가하지만, 가중치를 수정하는 대신 이전 예측기가 만든 **잔여 오차**에 새로운 예측기를 학습시킨다.

결정 트리를 기반 예측기로 사용하는 간단한 회귀 문제를 풀어보자. **그레디언트 트리 부스팅=그레디언트 부스티드 회귀 트리**.

In [17]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

첫 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor 훈련시키자.

In [18]:
y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

세 번째 모델

In [19]:
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

세 개의 트리를 포함하는 앙상블 모델이 생겼다. 새로운 샘플에 대한 예측을 만드려면 모든 트리의 예측을 더하면 된다.

In [20]:
y_pred=sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

NameError: ignored

그림을 보면 트리가 앙상블에 추가될수록 앙상블의 예측이 점차 좋아지는 것을 확인할 수 있다.

사이킷런에서 GradientBoostingRegressor를 사용하면 간단하게 GBRT를 훈련시킬 수 있다.

In [21]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

learning_rate 매개변수가 각 트리의 기여 정도를 조절한다. 낮으면 많은 트리가 필요해지지만 성능은 좋아진다. **= 축소**.

최적의 트리 수를 찾기 위해서는 조기 종료 기법을 사용할 수 있다. 간단하게는 staged_predict()를 사용한다.

120개의 트리로 GBRT 앙상블을 훈련시키고 최적의 트리 수를 찾기 위해 각 훈련 단계에서 검증 오차를 측정해보자.

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val=train_test_split(X, y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors=[mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=64)

많은 수의 트리를 먼저 훈련시키고 최적의 수를 찾기 위해 살펴보는 대신 실제로 훈련을 중지하는 방법으로 조기 종료를 구현할 수도 있다.

연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춰보자.

In [23]:
gbrt=GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error=float('inf')
error_going_up=0
for n_estimators in range(1, 120):
  gbrt.n_estimators=n_estimators
  gbrt.fit(X_train, y_train)
  y_pred=gbrt.predict(X_val)
  val_error=mean_squared_error(y_val, y_pred)
  if val_error<min_val_error:
    min_val_error=val_error
    error_going_up=0
  else:
    error_going_up+=1
    if error_going_up==5:
      break #조기 종료

GB는 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있는 subsample 매개변수도 지원한다. 편향은 높아지지만 분산은 낮아지고 속도를 높인다. **확률적 그레디언트 부스팅. **

최적화된 그레디언트 부스팅으로 XGB가 유명.

In [25]:
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred=xgb_reg.predict(X_val)

자동 조기 종료도 할 수 있다.

In [26]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.37454
[1]	validation_0-rmse:0.31934
[2]	validation_0-rmse:0.28312
[3]	validation_0-rmse:0.24675
[4]	validation_0-rmse:0.22679
[5]	validation_0-rmse:0.21384
[6]	validation_0-rmse:0.20920
[7]	validation_0-rmse:0.20699
[8]	validation_0-rmse:0.20723


/usr/local/lib/python3.9/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[9]	validation_0-rmse:0.20799


##7.6 스태킹
앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수 대신 취합하는 모델을 훈련시킬 수는 없을까? 각각 다른 값을 예측하고 마지막 예측기(블렌더, 메타 학습기)가 이 예측을 입력으로 받아 최종 예측을 만드는 과정을 보자.

블렌더를 학습시키는 일반적인 방법은 홀드 아웃 세트를 사용하는 것. 훈련 세트를 두 개의 섭셋으로 나눠 첫 번째 섭셋은 첫 번째 레이어의 예측을 훈련시키기 위해 사용된다. 그 다음 첫 번째 레이어의 예측기를 사용해 두 번째(홀드아웃) 세트에 대한 예측을 만든다. 홀드 아웃 세트의 각 샘플에 대해 세 개의 예측값이 있다. 타깃값은 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만들 수 있다. 블렌더가 새 훈련 세트로 훈련된다.

이런 방식의 블렌더를 여러 개 훈련시키는 것도 가능하다. 블렌더만의 레이어가 만들어진다. 훈련 세트를 세 개의 셉셋으로 나누어 첫 번째 세트는 첫 번째 레이어를 훈련시키는데 사용하고 나머지 세트에도 각자 레이어를 훈련시키는데 사용한다. 이런식으로 반복해 작업이 끝나면 각 레이어를 차례대로 실행해서 새로운 샘플에 대한 예측을 만들 수 있다.

사이킷런에서 직접 지원하지는 않는다.